

<img src='images/gdd-logo.png' width='300px' align='right' style="padding: 15px">

# Decorators

A decorator is a **design pattern** in Python that allows a user to **add new functionality** to an existing object **without modifying its structure**. For example, easily adding the functionality to time how long a function takes to run, without having to rewrite it. 

They are usually called before the definition of a function you want to decorate. 

Functions in Python are ***first class citizens***. This means they support operations such as being:
- Passed as an argument
- Returned from another function
- Modified
- Assigned to a variable 

A function being a ***first class citizen*** is a fundamental concept to understand to fully understand Python decorators. 

Therefore, notebook is split into two sections:
<img src='images/python-paint.png' width='200px' align='right' style="padding: 15px">

**Decorator Pre-requisites**
- [Assigning Functions to Variables](#assigning)
- [Defining Functions Inside other Functions](#defining)
- [Passing Functions as Arguments to other Functions](#passing)
- [Functions Returning other Functions](#return)
- [Nested Functions have access to the Enclosing Function's Variable Scope](#nested)

**Creating Decorators**
- [Motivation for Decorators](#motivate)
- [Creating your first Decorator](#create)
- [Applying Multiple Decorators](#applying)
- [Accepting Arguments in Decorator Functions](#accepting)
- [Defining General Purpose Decorators](#defining)

[**Usecases of decorators**](#usecases)

<a id='assigning'></a>
## Decorator Pre-requisites

### Assigning Functions to Variables

Let's take a function, that will multiply a number by 2 and assign it to a variable. You can then use this variable to call the function.

In [ ]:
def times_two(number):
    return number * 2

multiply_two = times_two
multiply_two(5)

<a id='defining'></a>
### Defining Functions Inside other Functions
Next, let's define a function inside another function in Python. This becomes very relevant when creating and understanding decorators.

In [ ]:
def multiply_two(number):
    def times_two(number):
        return number * 2

    result = times_two(number)
    return result

multiply_two(11)

<a id='passing'></a>
### Passing Functions as Arguments to other Functions
Functions can also be passed as arguments to other functions. 

In [ ]:
def multiply_two(number):
    return number * 2

def function_call(function):
    number_to_multiply = 5
    return function(number_to_multiply)

function_call(multiply_two)

<a id='return'></a>
### Functions Returning other Functions
A function can also generate another function. 

In [ ]:
def hello_function():
    
    def say_hi():
        return "Hello!"
    
    return say_hi

hello = hello_function()
hello()

<a id='nested'></a>
### Nested Functions have access to the Enclosing Function's Variable Scope
Python allows a nested function to access the outer scope of the enclosing function. This is a critical concept in decorators -- this pattern is known as ***Closure***.

In [ ]:
def hello_function(name='World'):
    "Enclosing Function"
    
    def say_hi():
        "Nested Function"
        print(f"Hello, {name}!")
    
    say_hi()

hello_function()

---
<a id='motivate'></a>
## Creating Decorators
### Motivation

Okay, now that you revisited the operations that you can perform on functions, lets look at how this ties in with Python Decorators:

Below is a simple function that returns some text.

In [ ]:
def say_hi():
    return 'hello, world!'
say_hi()

Let's imagine you wanted the option of returning the text in upper case. 

Of course, one option would be to simply rewrite our function. 

In [ ]:
def say_hi(upper=False):
    if upper:
        return 'hello, world!'.upper()
    else:
        return 'hello, world!'
say_hi(upper=True)

Alternatively, you could add this functionality by creating a decorator! This has the benefit of leaving our original function unchanged. Furthermore, it would allow you to add the functionality to other functions, too!

<a id='create'></a>
### Creating your first Decorator

You will now create a simple decorator that can convert the output of a function to uppercase.

How do you do this?
 1. The decorator function takes a **function as an argument.**
 2. You then create a **wrapper function**, which wraps around the function you passed and modifies its inputs or outputs.
 3. Lastly, the decorator function **returns the wrapper function**.

In [ ]:
def uppercase_decorator(function):
    
    def wrapper():
        return function().upper()

    return wrapper

Notice that the decorator takes a function as an argument and returns a different function!

To modify your function, you could assign the decorator function that uses the say_hi as input to a variable that you could then call later on.

In [ ]:
def say_hi():
    return 'hello, world!'

decorate = uppercase_decorator(say_hi)
decorate()

However, Python provides a much easier way for you to apply decorators. You simply use the `@` symbol before the function you would like to decorate. Let's show that in practice below.

In [ ]:
@uppercase_decorator
def say_hi():
    return 'hello, world!'

say_hi()

### <mark>Exercise: Create a "Split String" decorator</mark>

Create a decorator that would change a string into a list of words and test it on the `say_hi` function.

In [ ]:
# %load answers/ex-decorators1.py

<mark>**Bonus:**</mark> Create a decorator named `time_it` that prints the number of seconds a function needed to be executed.

*Hint: Import the `time` package from base Python and use `time.time()` before and after the function execution to calculate a time difference.*

In [ ]:
# %load answers/ex-decorators1-bonus.py

<a id='applying'></a>
### Applying Multiple Decorators
You can also apply multiple decorators to a single function. 

Let's apply both of our decorators to the `say_hi` function being mindful about order.

In [ ]:
# @split_string
# @uppercase_decorator
def say_hi():
    return 'hello, world'

say_hi()

<mark>***Questions:***</mark>

1. In what order do the decorators get applied? (top down or bottom up?)

2. Why can't you reverse the order of these decorators?

<details>
    
  <summary><span style="color:blue">Show answers</span></summary>
  
1. First, the `uppercase_decorator` is applied and then the `split_string` decorator.
2. Because the `uppercase_decorator` applies the `.upper()` method which does not work on lists (which you would have, if you applied `split_string` first). 
    
</details>

<a id='accepting'></a>

### Accepting Arguments in Decorator Functions
Sometimes, you might need to define a decorator that accepts arguments. You achieve this by **passing the arguments to the wrapper** function. You can then pass these arguments to the function that is being decorated at call time.

In [ ]:
# @uppercase_decorator
def cities(city_one, city_two):
    return f"I travelled from {city_one} to {city_two}"

cities("Amsterdam", "New York")

In [ ]:
def decorator_with_arguments(function):
    
    def wrapper_accepting_arguments(arg1, arg2):
        print(f"My arguments are: {arg1}, {arg2}")
        result = function(arg1, arg2)
        return result
        
    return wrapper_accepting_arguments


@decorator_with_arguments
def cities(city_one, city_two):
    return f"I travelled from {city_one} to {city_two}"

cities("Amsterdam", "New York")

<a id='defining'></a>

### Defining General Purpose Decorators
To define a general purpose decorator that can be applied to any function, you can use **positional arguments** ("args") and **keyword arguments** ("kwargs"). These `*args` and `**kwargs` collect all positional and keyword arguments and store them in the `args` (tuple) and `kwargs` (dictionary) variables. They allow you to pass as many arguments as you would like during function calls.

Let's create `a_decorator_passing_arbitrary_arguments` & `a_wrapper_accepting_arbitrary_arguments`.

In [ ]:
def a_decorator_passing_arbitrary_arguments(function):
    
    def a_wrapper_accepting_arbitrary_arguments(*args, **kwargs):
        print('Positional arguments:', args)
        print('Keyword arguments:', kwargs)
        result = function(*args, **kwargs)
        return result
    
    return a_wrapper_accepting_arbitrary_arguments

@a_decorator_passing_arbitrary_arguments
def function_with_no_argument():
    return "No arguments here."

function_with_no_argument()

Let's see how you would use the decorator using positional arguments.

In [ ]:
@a_decorator_passing_arbitrary_arguments
def function_with_arguments(a, b, c):
    print(a, b, c)

function_with_arguments(1,2,3)

Keyword arguments are passed using keywords. An illustration of this is shown below.

In [ ]:
@a_decorator_passing_arbitrary_arguments
def function_with_keyword_arguments(first_name, last_name):
    print(f"This has shown keyword arguments: {first_name} {last_name}")

function_with_keyword_arguments(first_name="Derrick", last_name="Mwiti")

### Retaining a function's original documentation

Let's look at this cities example...

In [ ]:
def decorator_with_arguments(function):
    
    def wrapper_accepting_arguments(*args, **kwargs):
        """Wrapper function accepting arguments"""
        print('Positional arguments:', args)
        print('Keyword arguments:', kwargs)
        result = function(*args, **kwargs)
        return result
    
    return wrapper_accepting_arguments


@decorator_with_arguments
def cities(city_one, city_two):
    """Print sentence of cities travelled"""
    
    return f"I travelled from {city_one} to {city_two}"

cities(city_one="Amsterdam", city_two="New York")

When you wrap a decorator around a function, the function inherits the documentation from the decorator:

In [ ]:
cities.__name__, cities.__doc__

If using a decorator always meant losing this information about a function, it would be a serious problem. Luckily, there is a function called  `functools.wraps` which is itself a decorator. This takes a function used in a decorator and adds the functionality of copying over the function name, docstring, arguments list, etc. 

In [ ]:
from functools import wraps

def decorator_with_arguments(function):
    
    @wraps(function)
    def wrapper_accepting_arguments(*args, **kwargs):
        """Wrapper function accepting arguments"""
        print('Positional arguments:', args)
        print('Keyword arguments:', kwargs)
        result = function(*args, **kwargs)
        return result
    
    return wrapper_accepting_arguments


@decorator_with_arguments
def cities(city_one, city_two):
    """Print sentence of cities travelled"""
    return f"I travelled from {city_one} to {city_two}"

cities(city_one="Amsterdam", city_two="New York")

Now, the `cities` function keeps the original name and docstring!

In [ ]:
cities.__name__, cities.__doc__

<a id='usecases'></a>
## Usecases of decorators

Decorators are widely used within libraries that provide some extensions designed to be used with user-defined functions.

Some common examples include:
* Defining fixtures for testing
* Wraping functions with loggers for monitoring
* Benchmarking functions
* Using Python functions to define external interfaces (e.g. Typer, FastAPI)

### <mark>Exercises</mark>

Here is a function `get_factors` that returns a list of factors for an input number.

*Note: Factors of a number are defined as numbers that divide the original number evenly or exactly. E.g. `[1,2,3,6]` for the number `6`.*

In [ ]:
def get_factors(n):
    "Return the factors of n." 
    factors = [x for x in range(1, (n+1))
               if n % x == 0] 
    return factors

get_factors(6)

★ Create a decorator that prints the positional/keyword arguments and wrap it around `get_factors`.

★★ Create a decorator that prints the name of the function and use `wraps` from functools to retain the name of the function. Check it using `get_factors.__name__`!

★★★ Create a decorator `is_prime` that also checks whether a number is a prime number or not. Use (and if necessary modify) the `time_it` decorator from before to check how long this takes for bigger numbers. 

**Hint:** A number is prime if it only has two factors: The number itself (`n`) and `1`.

**Answers:**

In [ ]:
# %load answers/ex-decorators2.1.py

In [ ]:
# %load answers/ex-decorators2.2.py

In [ ]:
# %load answers/ex-decorators2.3.py

<mark>**Bonus:**</mark> Investigate how to create decorators that accept arguments themselves.

For example, decorator that either lowercases or uppercases the output of another function based on a parameter.

```python 
@change_case(upper = True)
def say_hi():
    return "Hello!"

say_hi("people")
>>> "HELLO PEOPLE!"
```

In [ ]:
# %load answers/ex-decorators2-bonus.py